# Урок 5. Рекуррентные нейронные сети

## Практическое задание

<ol>
    <li>Попробуйте изменить параметры нейронной сети работающей с датасетом imdb либо нейронной сети работающей airline-passengers(она прилагается вместе с датасетом к уроку в виде отдельного скрипта) так, чтобы улучшить ее точность. Приложите анализ.</li>
    <li>Попробуйте изменить параметры нейронной сети генерирующий текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший получившейся у вас текст и опишите, что вы предприняли, чтобы его получить. Можно использовать текст другого прозведения.</li>
    <li>* Попробуйте на numpy реализовать нейронную сеть архитектуры LSTM</li>
    <li>* Предложите свои варианты решения проблемы исчезающего градиента в RNN</li>
</ol>

## Задание 1. Решение:

In [4]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Dropout, LSTM, ConvLSTM1D
from keras.datasets import imdb
from keras.utils import pad_sequences
from random import shuffle, choice
from tools import paralell_execution, ar_split_eq_cpu

max_features = 10000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 100
batch_size = 128 # увеличьте значение для ускорения обучения
activations = ['relu', 'LeakyReLU', 'elu']

# комбинации параметров нейросети 
best_acc = 0 # точность лучшей модели
best_params = {} # параметры лучшей модели
best_model = None # лучшая модель
params = []
for genf in [LSTM]: # LSTM, ConvLSTM1D
    for optimizer in ['SGD', 'adam']: # 'SGD','RMSProp','adam','NAdam'
        for loops in [1, 2]:
            for dense_size in [32, 64]:
                for epochs in [10]:
                    for dropout in [True, False]:
                        params.append({
                                           'genf': genf, 
                                           'optimizer': optimizer, 
                                           'dense_size': dense_size, 
                                           'epochs': epochs, 
                                           'dropout': dropout, 
                                           'activation': choice(activations), 
                                           'loops': loops, 
                                      })

shuffle(params)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

print(f'Всего комбинаций параметров: {len(params)}')

Всего комбинаций параметров: 16


In [2]:
def train(params):
    p = params['PARAMS']
    param = p['param']
    x_train = p['x_train']
    y_train = p['y_train']
    x_test = p['x_test']
    y_test = p['y_test']
    batch_size = p['batch_size']
    max_features = p['max_features']
    
    # print('Построение модели...')
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(param['genf'](128, dropout=0.4, recurrent_dropout=0.4))
    for i in range(param['loops']):
        model.add(Dense(param['dense_size'], activation=param['activation']))
        if param['dropout']:
            model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    # стоит попробовать использовать другие оптимайзер и другие конфигурации оптимайзеров
    model.compile(loss='binary_crossentropy',
                optimizer=param['optimizer'], # при использовании этого оптимайзера модель показывает наилучшие результаты.
                metrics=['accuracy'])
    
    # print('Процесс обучения...')
    model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=10, # увеличьте при необходимости
                validation_data=(x_test, y_test),
                verbose=0)
    
    score, acc = model.evaluate(x_test, 
                                y_test, 
                                batch_size=batch_size,
                                verbose=0)
    
    return {'best_acc': acc,
            'best_params': param,
            'model': model}


args_parallel = []
for param in ar_split_eq_cpu(params):
    args_parallel.append({
        'PARAMS':{
            'param': param[0],
            'x_train': x_train,
            'y_train': y_train,
            'x_test': x_test,
            'y_test': y_test,
            'batch_size': batch_size,
            'max_features': max_features,
        }
    })

# запускаем процесс параллельных вычислений
res = paralell_execution(func=train,
                         arg=args_parallel,
                         method='multiprocessing')

for row in res: # объединяем результаты со всех ядер
    if row['best_acc'] > best_acc:
        best_acc = row['best_acc']
        best_params = row['best_params']
        best_model = row['model']

print(f'Лучшая точность: {best_acc} при параметрах {best_params}')

Лучшая точность: 0.8278800249099731 при параметрах {'genf': <class 'keras.layers.rnn.lstm.LSTM'>, 'optimizer': 'adam', 'dense_size': 64, 'epochs': 10, 'dropout': False, 'activation': 'relu', 'loops': 2}


In [3]:
# Структура лучшей модели
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,424,065
Trainable params: 1,424,065
Non-trainable params: 0
_________________________________________________________________


### Ответ

Лучшая точность: 

0.8278800249099731 

При параметрах {'genf': <class 'keras.layers.rnn.lstm.LSTM'>, 'optimizer': 'adam', 'dense_size': 64, 'epochs': 10, 'dropout': False, 'activation': 'relu', 'loops': 2}

## Задание 2. Решение:

In [1]:
# alice_in_wonderland пример из урока

from random import shuffle, choice
from tools import paralell_execution, ar_split_eq_cpu
import numpy as np
import os
# Загрузка keras перенесена внутрь процесса. Иначе не работает при многоядерности: https://stackoverflow.com/questions/42504669/keras-tensorflow-and-multiprocessing-in-python
# from keras.layers import Dense, Activation
# from keras.layers import SimpleRNN, LSTM, GRU
# from keras.models import Sequential

# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)

# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# комбинации параметров нейросети 
best_acc = 0 # точность лучшей модели
best_params = {} # параметры лучшей модели
best_model = None # лучшая модель
params = []
for NUM_ITERATIONS in [15, 25]:
    for HIDDEN_SIZE in [64, 128]:
        for BATCH_SIZE in [64, 128]:
            for NUM_EPOCHS_PER_ITERATION in [1, 2]:
                for NUM_PREDS_PER_EPOCH in [100, 200]:
                    for algo in ['GRU', 'LSTM', 'SimpleRNN']: # LSTM, SimpleRNN
                        params.append({
                                           'NUM_ITERATIONS': NUM_ITERATIONS,
                                           'HIDDEN_SIZE': HIDDEN_SIZE,
                                           'BATCH_SIZE': BATCH_SIZE,
                                           'NUM_EPOCHS_PER_ITERATION': NUM_EPOCHS_PER_ITERATION,
                                           'NUM_PREDS_PER_EPOCH': NUM_PREDS_PER_EPOCH,
                                           'algo': algo,
                                      })
shuffle(params)
print(f'Всего комбинаций параметров: {len(params)}')

Всего комбинаций параметров: 96


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
def train(params):
    # Загрузка keras перенесена внутрь процесса. Иначе не работает при многоядерности: https://stackoverflow.com/questions/42504669/keras-tensorflow-and-multiprocessing-in-python
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    from keras.layers import Dense, Activation
    from keras.layers import SimpleRNN, LSTM, GRU
    from keras.models import Sequential
    
    p = params['PARAMS']
    param = p['param']
    X = p['X']
    y = p['y']
    SEQLEN = p['SEQLEN']
    STEP = p['STEP']
    text = p['text']
    chars = p['chars']
    nb_chars = p['nb_chars']
    input_chars = p['input_chars']
    label_chars = p['label_chars']
    char2index = p['char2index']
    index2char = p['index2char']
    
    # установка ряда метапамертров  для нейронной сети и процесса тренировки
    algo = param['algo'] 
    
    if algo == 'GRU':
        algo = GRU
    if algo == 'LSTM':
        algo = LSTM
    if algo == 'SimpleRNN':
        algo = SimpleRNN
        
    BATCH_SIZE = param['BATCH_SIZE']
    HIDDEN_SIZE = param['HIDDEN_SIZE']
    NUM_ITERATIONS = param['NUM_ITERATIONS'] # 25 должно быть достаточно
    NUM_EPOCHS_PER_ITERATION = param['NUM_EPOCHS_PER_ITERATION']
    NUM_PREDS_PER_EPOCH = param['NUM_PREDS_PER_EPOCH']
    
    # Создание очень простой рекуррентной нейронной сети. В ней будет один 
    # реккурентный закодированный входной слой. За ним последует полносвязный 
    # слой связанный с набором возможных следующих символов, которые 
    # конвертированы в вероятностные результаты через стандартную softmax 
    # активацию с multi-class cross-encoding loss функцию ссылающуются на 
    # предсказание one-hot encoding лейбл символа
    
    model = Sequential()
    model.add(
        algo(
            HIDDEN_SIZE,
            return_sequences=False,
            input_shape=(SEQLEN, nb_chars),
            unroll=True
        )
    )
    model.add(Dense(nb_chars))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    
    # выполнение серий тренировочных и демонстрационных итераций 
    for iteration in range(NUM_ITERATIONS):
        model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION, verbose=0)
    
    y_pred_seed = ''
    y_pred_final = ''
    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    # print("Генерация из посева: %s" % (test_chars))
    # print(test_chars, end="")
    y_pred_seed = test_chars
    
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        # print(y_pred, end="")
        y_pred_final += y_pred

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

    return {'phrase': f'Фраза из слов ({y_pred_seed}): {y_pred_final}',
            'param': param,
            'model': model}


args_parallel = []
for param in ar_split_eq_cpu(params):
    args_parallel.append({
        'PARAMS':{
            'param': param[0],
            'X': X,
            'y': y,
            'SEQLEN': SEQLEN,
            'STEP': STEP,
            'text': text,
            'chars': chars,
            'nb_chars': nb_chars,
            'input_chars': input_chars,
            'label_chars': label_chars,
            'char2index': char2index,
            'index2char': index2char,
        }
    })

# запускаем процесс параллельных вычислений
res = paralell_execution(func=train,
                         arg=args_parallel,
                         method='multiprocessing')

for row in res: # объединяем результаты со всех ядер
    phrase = row['phrase']
    param = row['param']
    print(f'Параметры: {param}')
    print(f'{phrase}')

Параметры: {'NUM_ITERATIONS': 15, 'HIDDEN_SIZE': 128, 'BATCH_SIZE': 64, 'NUM_EPOCHS_PER_ITERATION': 2, 'NUM_PREDS_PER_EPOCH': 200, 'algo': 'LSTM'}
Фраза из слов ( work to p): roject gutenberg literary archive foundation and distributing project gutenberg literary archive foundation and distributing project gutenberg literary archive foundation and distributing project gute
Параметры: {'NUM_ITERATIONS': 25, 'HIDDEN_SIZE': 64, 'BATCH_SIZE': 64, 'NUM_EPOCHS_PER_ITERATION': 1, 'NUM_PREDS_PER_EPOCH': 100, 'algo': 'SimpleRNN'}
Фраза из слов ( work to p): urstented the door and she had as she had as she had as she had as she had as she had as she had as 
Параметры: {'NUM_ITERATIONS': 15, 'HIDDEN_SIZE': 128, 'BATCH_SIZE': 128, 'NUM_EPOCHS_PER_ITERATION': 2, 'NUM_PREDS_PER_EPOCH': 100, 'algo': 'SimpleRNN'}
Фраза из слов ( work to p): roved the mouse was the mouse was the mouse was the mouse was the mouse was the mouse was the mouse 
Параметры: {'NUM_ITERATIONS': 25, 'HIDDEN_SIZE': 64, 'BATCH_SI